# GPT-2 117M Fine Tuning

Adaptation of https://github.com/ak9250/gpt-2-colab

Includes code for building a single huge English training text file based on Project Gutenberg (books) or Access to Insight (Dharma texts). ATI seems to overfit quite rapidly since it's only 6 MB, but a 99 MB subset of Gutenberg came out quite convincing after only 4 hours of training. Your mileage may vary.

## Setup

1) Make sure GPU is enabled, go to edit->notebook settings->Hardware Accelerator GPU

2) make a copy to your google drive, click on copy to drive in panel

Note: colab will reset after 12 hours make sure to save your model checkpoints to google drive around 10-11 hours mark or before, then go to runtime->reset all runtimes. Now copy your train model back into colab and start training again from the previous checkpoint.

### Initialize

clone and cd into repo

In [1]:
!git clone https://github.com/nshepperd/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 212, done.
remote: Total 212 (delta 0), reused 0 (delta 0), pack-reused 212
Receiving objects: 100% (212/212), 4.37 MiB | 13.85 MiB/s, done.
Resolving deltas: 100% (112/112), done.


install requirements

In [3]:
!pip3 install --upgrade tensorflow-gpu beautifulsoup4
!pip3 install -r requirements.txt

    100% |████████████████████████████████| 345.2MB 49kB/s 
    100% |████████████████████████████████| 102kB 27.4MB/s 
fastai 1.0.51 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
    100% |████████████████████████████████| 604kB 13.2MB/s 
    100% |████████████████████████████████| 61kB 21.7MB/s 
    100% |████████████████████████████████| 51kB 13.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
spacy 2.0.18 has requirement regex==2018.01.10, but you'll have regex 2017.4.5 which is incompatible.
google-colab 1.0.0 has

In [2]:
cd gpt-2

/content/gpt-2


download the model

In [3]:
!python3 download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 693kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 35.8Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 973kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:10, 48.7Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 4.76Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 18.3Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 33.3Mit/s]                                                       


set encoding

In [0]:
!export PYTHONIOENCODING=UTF-8

### Mount Google Drive

mount drive to access google drive for saving and accessing checkpoints later

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


(optional) fetch checkpoints if you have them saved in google drive

In [0]:
!cp -r /content/drive/My\ Drive/checkpoint/ /content/gpt-2/ 

## Download + Prepare Training Data

In [6]:
cd /content/gpt-2

/content/gpt-2


In [0]:
mkdir data

In [8]:
cd data

/content/gpt-2/data


### Project Gutenberg

Download through their [Robot Access](http://www.gutenberg.org/wiki/Gutenberg:Information_About_Robot_Access_to_our_Pages)

(this will take a while)

/content/gpt-2/data


In [0]:
!wget -w 0.5 -m -H "http://www.gutenberg.org/robot/harvest?filetypes[]=txt&langs[]=en"

--2019-04-05 22:12:12--  http://www.gutenberg.org/robot/harvest?filetypes[]=txt&langs[]=en
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13052 (13K) [text/html]
Saving to: ‘www.gutenberg.org/robot/harvest?filetypes[]=txt&langs[]=en’

www.gutenberg.org/r 100%[===================>]  12.75K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2019-04-05 22:12:12 (258 MB/s) - ‘www.gutenberg.org/robot/harvest?filetypes[]=txt&langs[]=en’ saved [13052/13052]

Loading robots.txt; please ignore errors.
--2019-04-05 22:12:13--  http://aleph.gutenberg.org/robots.txt
Resolving aleph.gutenberg.org (aleph.gutenberg.org)... 65.50.255.20, 2604:3200:0:3:1618:77ff:fe49:8a7
Connecting to aleph.gutenberg.org (aleph.gutenberg.org)|65.50.255.20|:80... connected.
HTTP request sent, awaiting r

In [0]:
!find . -name "*[!-8].zip" | while read filename; do unzip -o -d "`basename -s .zip "$filename"`" "$filename"; done;

omit this?

In [0]:
!find . -name "*.txt" | while read filename; do cp $filename /content/gpt-2/books/; done;

In [0]:
!find . -name "*.txt" | while read filename; do iconv -f ascii -t utf8 $filename > $filename-utf8.txt ; done;

In [0]:
cat *utf8.txt >> allbooks-utf8.txt

### Access to Insight

Bulk download from [this page](https://accesstoinsight.org/tech/download/bulk.html)

In [0]:
!wget "http://accesstoinsight.org/tech/download/ati.zip"

unzip the archive

In [0]:
!unzip ati.zip

In [0]:
cd /content/gpt-2/data/ati

#### Convert HTML to Text

Parser for Access to Insight HTML dump based on [this script](https://codereview.stackexchange.com/questions/128515/parsing-locally-stored-html-files) and [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/index.html)

In [0]:
from bs4 import BeautifulSoup
import glob
import os
import re
import contextlib


@contextlib.contextmanager
def stdout2file(fname):
    import sys
    f = open(fname, 'w')
    sys.stdout = f
    yield
    sys.stdout = sys.__stdout__
    f.close()


def parser():
    # os.chdir(r"/contents/gpt-2/data/ati/)
    with stdout2file("dhamma_cleaned.txt"):
        for file in glob.iglob('tipitaka/**/*.html', recursive=True):
            with open(file, encoding="utf8") as f:
                contents = f.read()
                soup = BeautifulSoup(contents, "html.parser")
                for item in soup.find_all(["blockquote","h4","p"]):
                    print(item.get_text())
                    print('\n')
                # break
parser()

## Train the Model

enter the directory

In [2]:
cd gpt-2

/content/gpt-2


initiate training (set to data file created in previous step)

In [0]:
!PYTHONPATH=src ./train.py --dataset /content/gpt-2/data/dhamma_cleaned.txt

save our checkpoints to start training again later

In [0]:
!cp -r /content/gpt-2/checkpoint/ /content/drive/My\ Drive/

copy re-trained (fine-tuned) model into the main directory

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

## Use the Trained Model

There are a few flags available, with a default value:

* `seed = None` || a random value is generated unless specified. give a specific integer value if you want to reproduce same results in the future.
* `nsamples = 1` || specify the number of samples you want to print
* `length = None` || number of tokens (words) to print on each sample.
* `batch_size= 1` || how many inputs you want to process simultaneously. doesn't seem to affect the results.
* `temperature = 1` || scales logits before sampling prior to softmax.
* `top_k = 0` || truncates the set of logits considered to those with the highest values.

### Conditional samples

In [5]:
!python3 /content/gpt-2/src/interactive_conditional_samples.py --top_k=40 --nsamples=3 --temperature=0.7 --length=100

2019-04-05 21:24:52.846808: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-04-05 21:24:52.950978: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-04-05 21:24:52.951626: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x17c3b80 executing computations on platform CUDA. Devices:
2019-04-05 21:24:52.951670: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla K80, Compute Capability 3.7
2019-04-05 21:24:52.953964: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-05 21:24:52.954201: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x17c3e40 executing computations on platform Host. Devices:
2019-04-05 21:24:52.954244: I tensorflow/compiler/

### Unconditional samples

In [0]:
!python3 src/generate_unconditional_samples.py | tee /tmp/samples